In [ ]:
import pandas as pd
import numpy as np

#BACKTESTING
#Inicialización de parámetros
UP = 2
DOWN = -2
PROFIT = 0
df_par['Posición'].iloc[0]  = 'FLAT' 
Posición = 'FLAT'
Trade = 'NTG'

#Función para calcular la posición actual y el trade que debe hacerse en cada momento
def calcular_posicion(Cociente):  
        
    global Posición
    global Trade     
    
    if Posición == 'FLAT':        
        if Cociente > UP:
            Trade = 'SELL'
            Posición = 'SHORT'
        elif Cociente < DOWN:
            Trade = 'BUY'
            Posición = 'LONG'
        else: 
            Trade = 'NTG'
            Posición = 'FLAT'
    
    elif Posición == 'SHORT':
        if Cociente < PROFIT:
            Trade = 'BUY'
            Posición = 'FLAT'
        else: 
            Trade = 'NTG'
            Posición = 'SHORT'
    
    elif Posición == 'LONG':    
        if Cociente > PROFIT:
            Trade = 'SELL'
            Posición = 'FLAT'
        else: 
            Trade = 'NTG'
            Posición = 'LONG'       
        
    return Posición, Trade

#Asignando los resultados de la función calcular_posicion al dataframe
resultado = df_par.apply(lambda row: calcular_posicion(Cociente=row['Cociente']),axis=1)
df_par['Posición'] = [x[0] for x in resultado]
df_par['Trade'] = [x[1] for x in resultado]

#Calculando los retornos de los trades en los que se entraron
df_trades = df_par[df_par['Posición'].isin(['LONG'])|df_par['Posición'].isin(['SHORT'])|df_par['Trade'].isin(['BUY'])|df_par['Trade'].isin(['SELL'])]  #dejamos solo las filas cuando hay posición
df_trades['retorno'] = df_trades['Log_ret'][:-1].values-df_trades['Log_ret'][1:]
conditions = [ (df_trades['Posición'] == 'SHORT') , (df_trades['Posición'] == 'LONG'), (df_trades['Posición'] == 'FLAT')&(df_trades['Trade'] == 'BUY'),(df_trades['Posición'] == 'FLAT')&(df_trades['Trade'] == 'SELL')]     
choices = [1.0,-1.0,1.0,-1.0]
df_trades['retorno_signo'] = np.select(conditions, choices, default='null')
df_trades['retorno_signo'] = df_trades['retorno_signo'].astype(float)
df_trades.loc[(df_trades.Posición == 'SHORT')&(df_trades.Trade == 'SELL')|(df_trades.Posición == 'LONG')&(df_trades.Trade == 'BUY'),'retorno']=0
df_trades['retorno_disc'] = np.exp(df_trades['retorno']*df_trades['retorno_signo'])-1
df_trades['retorno_acum'] = df_trades['retorno_disc'].cumsum()
